In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [4]:
from dotenv import load_dotenv
load_dotenv("../.env")

False

In [6]:
from crewai_tools import (
    FileReadTool,
    ScrapeWebsiteTool,
    MDXSearchTool,
    SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path="./fake_resume.md")
semantic_search_resume = MDXSearchTool(mdx="./fake_resume.md")

Inserting batches in chromadb: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.77it/s]


In [7]:
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on job posting to "
         "help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in navigating "
        "and extracting critical information "
        "from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought by employers, "
        "forming the foundation for effective application tailoring."
    )
)

In [8]:
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
        "to help them stand out in the job market",
    tools = [scrape_tool,
             search_tool,
             read_resume,
             semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, "
        "you dissect and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, "
        "laying the groundwork for personalized resume enhancements."
    )
)

In [9]:
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make "
         "a resume stand out in the job market.",
    tools = [scrape_tool,
             search_tool,
             read_resume,
             semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, "
        "you excel at refining resumes to highlight "
        "the most relevant skills and experiences, "
        "ensuring they resonate perfectly with "
        "the job's requirements."
    )
)

In [15]:
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools=[scrape_tool,
           search_tool,
           read_resume,
           semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating "
        "the dynamics of interviews."
        "With your ability to formulate key questions "
        "and talking points, "
        "you prepare candidates for success, "
        "ensuring they can confidently "
        "address all aspects of the job they are applying for."
    )
)

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as context to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task], # 
    agent=resume_strategist
)

In [16]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [17]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],
    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
    verbose=True
)

In [18]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [19]:
### this execution will take some time to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach.). Utilize tools to extract and synthesize information from these sources.

## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualification

In [20]:
from IPython.display import Markdown
Markdown(result.raw)

### Interview Questions and Talking Points:

#### Experience and Skills:
1. Can you discuss your experience as a Full Stack Developer and how it aligns with the requirements of this role?
2. How have you applied your knowledge of front-end languages and libraries, as well as object-oriented languages and frameworks, in your previous projects?
3. Describe a project where you showcased your expertise in developing both desktop and mobile applications.
4. How have you managed databases, web servers, and UI/UX design in your previous roles, and what challenges did you overcome?
5. Provide examples of your project management and organizational skills in the tech industry.

#### Education and Certifications:
1. How has your educational background in Computer Science and your MBA contributed to your success in tech leadership roles?
2. Can you explain your understanding of AI/ML models and big data tools, and how you have implemented them in your projects?

#### Communication and Team Management:
1. How do you approach technical discussions with both engineering teams and business stakeholders?
2. Share an experience where you successfully led remote and in-office teams, emphasizing your communication and leadership style.
3. How have you demonstrated a self-starter mentality in early-stage tech environments?

#### Projects and Achievements:
1. Discuss your experience in software engineering leadership and strategic planning, drawing from specific projects and achievements.
2. How did you leverage data science and AI in your projects at organizations like DataKernel and InnovPet?
3. Provide examples of open-source contributions that highlight your expertise and passion for tech innovation.

#### Personal Interests and Future Goals:
1. How do your interests in tech entrepreneurship and collaborative intelligence align with your career goals?
2. Where do you see yourself in terms of leading tech initiatives and startups in the future?
3. How do you plan to continue growing and innovating in the tech industry?

#### Additional Points:
1. How do you stay updated with the latest trends and technologies in the tech industry?
2. Can you discuss a challenging project you worked on and how you overcame obstacles to achieve success?
3. In what ways do you ensure compliance with data privacy regulations and best practices in your projects?

By preparing for these questions and points, you can confidently showcase your relevant experiences, skills, and aspirations during the initial interview, highlighting your suitability for the role.